In [55]:
#ライブラリのインポート
import tkinter as tk
import cv2
import PIL.Image, PIL.ImageTk
from PIL import Image, ImageTk, ImageOps
import json
import os
import re

class Application(tk.Frame):
    
    ori_data_path = "./data/ori_data/"
    files_path = os.listdir(ori_data_path + 'drama_image')
    files = sorted(files_path, key=lambda s: int(re.search('\d+', s).group()))
    
    image_id = []
    item = ["1","2","3","4","5","6"]
    stage = ["1","2","3"]
    
    rect_list = []
    rect_id_list = []
    label_list = []
    txt_list = []
    check_list = []
    radio_list = []
    button_list = []
    risk_label_list = []
    risk_stage_list = []
    frame_list = []
    
    label_pos_list = []
    button_pos_list = []
    text_pos_list = []
    check_pos_list = []
    radio_pos_list = []
    risk_label_pos_list = []
    risk_stage_pos_list = []
    frame_pos_list = []
    
    all_check_var_list = []
    all_radio_var_list = []
    check_var_list = []
    radio_var_list = []
    check_var_dict = {}
    txt_var_list = []
    
    
    box_start_list = []
    box_end_list = []
    
    region_json_data = []
    image_json_data = []
    class_list = []
    
    counter_box = -1
    counter_img = 1
    region_id = 0
    remove_counter = 0
    
    
    
    def __init__(self, master=None):
        super().__init__(master)
        self.master.geometry("2000x1000")#ウィンドウサイズ
        #self.master.attributes('-fullscreen', False)
        self.master.title("OpenCV 動画表示")#タイトル設定
        
        if os.path.exists(self.ori_data_path + 'region_info.json') and os.path.exists(self.ori_data_path + 'image_info.json'):
            
            region_json_open = open(self.ori_data_path + 'region_info.json', 'r')
            image_json_open = open(self.ori_data_path + 'image_info.json', 'r')
            region_json_load = json.load(region_json_open)
            image_json_load = json.load(image_json_open)
        
            if region_json_load == None or image_json_load == None:
                #print("a")
                pass
            else:
                for i,one_region_data in enumerate(region_json_load):
                    self.region_json_data.append(one_region_data)#既存の領域データを変数に格納
                    
                    if i == len(region_json_load)-1:
                        self.counter_img = self.region_json_data[-1]["id"] + 1
                        self.region_id = self.region_json_data[-1]["regions"][-1]['region_id']
                    
                    
                for i,one_image_data in enumerate(image_json_load):
                    self.image_json_data.append(one_image_data)#既存の画像データを変数に格納
        
        #動画ファイルの読み込み
        self.cap = cv2.VideoCapture('C:/Users/naoki/anaconda_work/Faster-R-CNN-pytorch-main/data/ori_data/drama_clip/{}/clip.mp4'.format(self.counter_img - 1))
        
        #プロパティ
        self.width = self.cap.get( cv2.CAP_PROP_FRAME_WIDTH)
        self.height = self.cap.get( cv2.CAP_PROP_FRAME_HEIGHT)
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
        self.frame_num = self.cap.get(cv2.CAP_PROP_FRAME_COUNT)
        self.time = self.frame_num / self.fps
        print(self.fps,self.frame_num)
        self.create_frame()
        self.create_widget()
        self.create_table()
        self.create_stage_table()
        self.button_conf()
        self.delay = 10
        self.play_video()

    def create_frame(self):
        #フレームの作成
        self.frame1 = tk.Frame(self.master, width=1000, height=740, bg="#E6E6E6")
        self.frame1.place(x=20, y=20)
        self.frame2 = tk.Frame(self.master, width=640, height=820, relief=tk.RAISED, bd=5, bg="lightgrey")
        self.frame2.place(x=1025, y=20)
        self.frame3 = tk.Frame(self.master, width=600, height=20, relief=tk.RAISED, bd=3, bg="lightgrey")
        self.frame3.place(x=500, y=920)
        self.frame4 = tk.Frame(self.master, width=250, height=100, relief=tk.RAISED, bd=3, bg="lightgrey")
        self.frame4.place(x=1230, y=875)
        self.frame5 = tk.Frame(self.master, width=1000, height=160, relief=tk.RAISED, bd=3, bg="lightgrey")
        self.frame5.place(x=20, y=790)
        self.frame6 = tk.Frame(self.master, width=500, height=160, relief=tk.RAISED, bd=3, bg="lightgrey")
        self.frame6.place(x=500, y=790)
        
        # Canvasの作成
        self.canvas = tk.Canvas(self.frame1, width=1000, height=740,)
        self.anno_canvas = tk.Canvas(self.frame2, width=640, height=820,)
        # Canvasを配置
        self.canvas.pack()
        self.anno_canvas.pack()
        
    def create_widget(self):
        #Labelの生成
        l_property = tk.Label(self.frame3,text="プロパティ", relief="flat")
        l_property.grid(row=0, column=0, sticky = tk.W)
        l_id = tk.Label(self.frame3,text="image_id:{}".format(self.counter_img), relief="flat")
        l_id.grid(row=0, column=1, sticky = tk.W)
        l_width = tk.Label(self.frame3,text="width: "+str(int(self.width)), relief="flat")
        l_width.grid(row=0, column=2, sticky = tk.W)
        l_height = tk.Label(self.frame3,text="height: "+str(int(self.height)), relief="flat")
        l_height.grid(row=0, column=3, sticky = tk.W)
        l_fps = tk.Label(self.frame3,text="fps: "+str(int(self.fps)), relief="flat")
        l_fps.grid(row=0, column=4, sticky = tk.W)
        l_time = tk.Label(self.frame3,text="time: "+str(int(self.time))+"sec", relief="flat")
        l_time.grid(row=0, column=5, sticky = tk.W)
        
    def create_table(self):
        
        one_id = tk.Label(self.frame5,text="①", relief="flat", font=("MSゴシック", "15", "bold"), background='orange')
        one_id.grid(row=0, column=0, sticky = tk.W)
        one_risk = tk.Label(self.frame5,text="対象・障害物が自車に近い・近づく", relief="flat", font=("MSゴシック", "15", "bold"))
        one_risk.grid(row=0, column=1, sticky = tk.W)
        two_id = tk.Label(self.frame5,text="②", relief="flat", font=("MSゴシック", "15", "bold"), background='orange')
        two_id.grid(row=1, column=0, sticky = tk.W)
        two_risk = tk.Label(self.frame5,text="対象の動き・有無が予測できない。しにくい", relief="flat", font=("MSゴシック", "15", "bold"))
        two_risk.grid(row=1, column=1, sticky = tk.W)
        three_id = tk.Label(self.frame5,text="③", relief="flat", font=("MSゴシック", "15", "bold"), background='orange')
        three_id.grid(row=2, column=0, sticky = tk.W)
        three_risk = tk.Label(self.frame5,text="対象が自車進路に侵入するかもしれない", relief="flat", font=("MSゴシック", "15", "bold"))
        three_risk.grid(row=2, column=1, sticky = tk.W)
        four_id = tk.Label(self.frame5,text="④", relief="flat", font=("MSゴシック", "15", "bold"), background='orange')
        four_id.grid(row=3, column=0, sticky = tk.W)
        four_risk = tk.Label(self.frame5,text="対象が自車の存在に気づいていない", relief="flat", font=("MSゴシック", "15", "bold"))
        four_risk.grid(row=3, column=1, sticky = tk.W)
        five_id = tk.Label(self.frame5,text="⑤", relief="flat", font=("MSゴシック", "15", "bold"), background='orange')
        five_id.grid(row=4, column=0, sticky = tk.W)
        five_risk = tk.Label(self.frame5,text="対象が急停止する", relief="flat", font=("MSゴシック", "15", "bold"))
        five_risk.grid(row=4, column=1, sticky = tk.W)
        five_id = tk.Label(self.frame5,text="⑥", relief="flat", font=("MSゴシック", "15", "bold"), background='orange')
        five_id.grid(row=5, column=0, sticky = tk.W)
        five_risk = tk.Label(self.frame5,text="死角がある", relief="flat", font=("MSゴシック", "15", "bold"))
        five_risk.grid(row=5, column=1, sticky = tk.W)
    
    def create_stage_table(self):
        stage_label = tk.Label(self.frame6,text="リスク度", relief="flat", font=("MSゴシック", "15", "bold"), background='orange')
        stage_label.grid(row=0, column=0, sticky = tk.W)
        one_id = tk.Label(self.frame6,text="①", relief="flat", font=("MSゴシック", "15", "bold"), background='orange')
        one_id.grid(row=1, column=0, sticky = tk.W + tk.E)
        one_risk = tk.Label(self.frame6,text="進行方向にリスク対象が存在", relief="flat", font=("MSゴシック", "15", "bold"))
        one_risk.grid(row=1, column=1, sticky = tk.W)
        two_id = tk.Label(self.frame6,text="②", relief="flat", font=("MSゴシック", "15", "bold"), background='orange')
        two_id.grid(row=2, column=0, sticky = tk.W + tk.E)
        two_risk = tk.Label(self.frame6,text="進行方向以外にリスク対象が存在", relief="flat", font=("MSゴシック", "15", "bold"))
        two_risk.grid(row=2, column=1, sticky = tk.W)
        three_id = tk.Label(self.frame6,text="③", relief="flat", font=("MSゴシック", "15", "bold"), background='orange')
        three_id.grid(row=3, column=0, sticky = tk.W + tk.E)
        three_risk = tk.Label(self.frame6,text="死角にリスク対象が存在", relief="flat", font=("MSゴシック", "15", "bold"))
        three_risk.grid(row=3, column=1, sticky = tk.W)
    
    def button_conf(self):
        next_button = tk.Button(self.frame4, width=10, height = 5, text='NEXT', command=self.btn_click_next)#, command=self.btn_click_next)
        next_button.place(x=140, y=5)
        re_button = tk.Button(self.frame4, width=10, height = 5, text='もう一度再生', command=self.btn_click_re)#, command=self.btn_click_next)
        re_button.place(x=20, y=5)
        
    def play_video(self):
        #動画をリサイズしてキャンバスに表示
        
        ret, frame = self.cap.read()
        if ret == False:
            self.image_view()
            return
        
        self.current_frame = self.cap.get(cv2.CAP_PROP_POS_FRAMES)
        l_time = tk.Label(self.frame3,text="frame: "+str(int(self.current_frame)), relief="flat")
        l_time.grid(row=0, column=5, sticky = tk.W)

        frame = cv2.resize(frame, dsize=(1000, 740))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)        
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(frame))
        self.canvas.create_image(0,0, image= self.photo, anchor = tk.NW)
        #alter関数 10ms後に関数を実行
        self.master.after(self.delay, self.play_video)
        
    def image_view(self):#画像の表示
        
        self.image = Image.open("C:/Users/naoki/anaconda_work/Faster-R-CNN-pytorch-main/data/ori_data/drama_image/{}.jpg".format(self.counter_img))
        self.tk_img = ImageTk.PhotoImage(self.image)
        self.img_width, self.img_height = self.image.size
        self.canvas.create_image(0,0, image= self.tk_img, anchor = tk.NW)
        for i in range(len(self.rect_list)):
            if self.box_start_list[i] != None:
                self.canvas.create_rectangle(self.box_start_list[i][0], self.box_start_list[i][1], self.box_end_list[i][0], self.box_end_list[i][1], 
                                         outline = 'red', width = 5, tag="rect_{}".format(i))#四角を描画
        for i in range(len(self.rect_id_list)):
            if self.rect_id_list[i] != None:
                self.rect_id_list[i].place(x=self.box_start_list[i][0], y=self.box_start_list[i][1])
        self._createCanvasBinding()#操作開始self.box_start_list
        
    def _createCanvasBinding(self):
        
        """
        
        マウスクリックなどのイベントを識別して、イベントに応じた関数を呼び出す
        
        """
        
        self.canvas.bind( "<Button-1>", self.startRect )
        self.canvas.bind( "<ButtonRelease-1>", self.stopRect )
        self.canvas.bind( "<B1-Motion>", self.movingRect )

    def unbind(self):
        
        self.canvas.unbind("<Button-1>")
        self.canvas.unbind("<ButtonRelease-1>")
        self.canvas.unbind("<B1-Motion>")
    
    def btn_click(self, create_numbers):
        
        """
        
        ボックスに付与されている "取り消し" ボタンが押されたときの処理
        
        args
        ---------------
        
        create_numbers
            何番目に作成されたボックスかを表すID
        
        """
        
        def x():
            self.remove_counter += 1
            self.canvas.delete(self.rect_list[create_numbers])
            self.rect_list[create_numbers] = None
            self.rect_id_list[create_numbers].destroy()
            self.rect_id_list[create_numbers] = None
            self.label_list[create_numbers].destroy()
            self.label_list[create_numbers] = None
            self.risk_label_list[create_numbers].destroy()
            self.risk_label_list[create_numbers] = None
            self.risk_stage_list[create_numbers].destroy()
            self.risk_stage_list[create_numbers] = None
            self.anno_canvas.delete("frame_{}".format(create_numbers))
            
            
            for i in range(len(self.item)):
                
                self.check_list[create_numbers][i].destroy()
                self.check_list[create_numbers][i] = None
            
            for i in range(len(self.stage)):
                
                self.radio_list[create_numbers][i].destroy()
                self.radio_list[create_numbers][i] = None
                
            self.all_check_var_list[create_numbers] = None
            
            self.txt_list[create_numbers].destroy()
            self.txt_list[create_numbers] = None
            self.button_list[create_numbers].destroy()
            self.button_list[create_numbers] = None
            
            self.canvas.delete("rect_{}".format(create_numbers))
            self.box_start_list[create_numbers] = None
            self.box_end_list[create_numbers] = None
            
            self.check_var_list[create_numbers] = None
            self.radio_var_list[create_numbers] = None
            self.txt_var_list[create_numbers] = None
            
            
            for i in range(create_numbers+1,len(self.button_list)):
                if self.button_list[i] != None:
                    
                    self.button_list[i].place(x=self.button_pos_list[i][0],y=self.button_pos_list[i][1]-115)
                    self.button_pos_list[i][1] -= 115
                    
                    self.txt_list[i].place(x=self.text_pos_list[i][0],y=self.text_pos_list[i][1]-115)
                    self.text_pos_list[i][1] -= 115
                    
                    self.label_list[i].place(x=self.label_pos_list[i][0],y=self.label_pos_list[i][1]-115)
                    self.label_pos_list[i][1] -= 115
                    
                    self.risk_label_list[i].place(x=self.risk_label_pos_list[i][0],y=self.risk_label_pos_list[i][1]-115)
                    self.risk_label_pos_list[i][1] -= 115
                    
                    self.risk_stage_list[i].place(x=self.risk_stage_pos_list[i][0],y=self.risk_stage_pos_list[i][1]-115)
                    self.risk_stage_pos_list[i][1] -= 115
                    
                    self.anno_canvas.move("frame_{}".format(i), 0, -115)
                    self.frame_pos_list[i][1] -= 115
                    
                    
                    for l in range(len(self.item)):
                        #print(self.check_list[i][l])
                        
                        self.check_list[i][l].place(x=self.check_pos_list[i][l][0],y=self.check_pos_list[i][l][1]-115)
                        self.check_pos_list[i][l][1] -= 115
                        
                    for l in range(len(self.stage)):
                    
                        self.radio_list[i][l].place(x=self.radio_pos_list[i][l][0],y=self.radio_pos_list[i][l][1]-115)
                        self.radio_pos_list[i][l][1] -= 115
                    
        return x
    
    def startRect(self, event):
        
        """
        
        マウスを左クリックされたときに呼び出される関数
        
        args
        ---------------
        
        event : tkinter.Eventクラス
            左クリックしたイベントの情報が保持されたクラス
            event.x はクリックされたx座標
            event.y はクリックされたy座標
        
        """
        
        self.counter_box += 1
        
        self.rectx0 = self.canvas.canvasx(event.x)
        self.recty0 = self.canvas.canvasy(event.y) 
        
        self.rectid = self.canvas.create_rectangle(
            self.rectx0, self.recty0, self.rectx0, self.recty0, outline = 'red', width = 5)
        
        self.rect_list.append(self.rectid)
    
    def movingRect(self, event):
        
        """
        
        マウスが左クリックされた状態でのドラッグ動作の情報を取得し、情報をウィンドウに表示する。
        
        args
        ---------------
        
        event : tkinter.Eventクラス
            ドラッグ状態の情報が保持されたクラス
            event.x は現在のx座標
            event.y は現在のy座標
        
        """
        
        self.rectx1 = self.canvas.canvasx(event.x)
        self.recty1 = self.canvas.canvasy(event.y)
        
        self.canvas.coords(self.rectid, self.rectx0, self.recty0,
                      self.rectx1, self.recty1)
        
    def stopRect(self, event):
        
        """
        
        左クリックが離されたときの情報を取得し、各種処理を行う関数
        
        args
        ---------------
        
        event : tkinter.Eventクラス
            左クリックを離すというイベントの情報が保持されたクラス
            event.x は左クリックが離されたときのx座標
            event.y は左クリックが離されたときのy座標
        
        """
        
        
        
        self.rectx1 = self.canvas.canvasx(event.x)
        self.recty1 = self.canvas.canvasy(event.y)
        
        self.canvas.coords(self.rectid, self.rectx0, self.recty0,
                      self.rectx1, self.recty1)
        
        self.box_start_list.append([self.rectx0, self.recty0])
        self.box_end_list.append([self.rectx1, self.recty1])
        
        self.txt_var_list.append(tk.StringVar())
        self.txt_var_list[self.counter_box].set("")
        
        self.rect_id_list.append(tk.Label(self.frame1, text="ID : {}".format(self.counter_box), width=5))
        self.rect_id_list[self.counter_box].place(x=self.rectx0, y=self.recty0)
        
        one_box_check = []
        one_box_radio = []
        self.check_var_dict = {}
        #self.radio_var_dict = {}
        
        for i in range(len(self.item)):
            self.check_var_dict[i]=tk.BooleanVar()
            self.check_var_dict[i].set(False)
        self.check_var_list.append(self.check_var_dict)
        
        for i in range(len(self.item)):
            one_box_check.append(tk.Checkbutton(self.frame2,variable=self.check_var_list[self.counter_box][i],
                   text=self.item[i]))
        self.check_list.append(one_box_check)
        
        self.all_check_var_list.append(self.check_var_dict)
        
        #for i in range(len(self.stage)):
            #self.radio_var_dict[i]=tk.IntVar() 
            #self.radio_var_dict[i].set(0)
        self.radio_var_list.append(tk.IntVar())
        #self.radio_var_list[self.counter_box].set(0)
        
        for i in range(len(self.stage)):
            one_box_radio.append(tk.Radiobutton(self.frame2,variable=self.radio_var_list[self.counter_box],
                   text=self.stage[i],value=i))
        self.radio_list.append(one_box_radio)
        
        self.all_radio_var_list = self.radio_var_list
        
        one_c_pos = []
        for i in range(len(self.item)):
            c_x, c_y = 100 + i * 50, 100  + self.counter_box * 115
            one_c_pos.append([c_x, c_y - self.remove_counter * 115])
            self.check_list[self.counter_box][i].place(x=c_x, y=c_y - self.remove_counter * 115)
        self.check_pos_list.append(one_c_pos)
        
        one_stage_pos = []
        for i in range(len(self.stage)):
            stage_x, stage_y = 100 + i * 50, 50  + self.counter_box * 115
            one_stage_pos.append([stage_x, stage_y - self.remove_counter * 115])
            self.radio_list[self.counter_box][i].place(x=stage_x, y=stage_y - self.remove_counter * 115)
        self.radio_pos_list.append(one_stage_pos)
        
        l_x , l_y = 10, 65  + self.counter_box * 115
        self.label_pos_list.append([l_x , l_y - self.remove_counter * 115])
        self.label_list.append(tk.Label(self.frame2, text="領域ID : {}".format(self.counter_box), width=8))
        self.label_list[self.counter_box].place(x=l_x, y=l_y - self.remove_counter * 115)
        
        t_x , t_y = 450, 100  + self.counter_box * 115
        self.text_pos_list.append([t_x ,t_y - self.remove_counter * 115])
        self.txt_list.append(tk.Entry(self.frame2, width=20,textvariable = self.txt_var_list[self.counter_box]))
        self.txt_list[self.counter_box].place(x=t_x, y=t_y - self.remove_counter * 115)
        
        b_x ,b_y = 550, 10  + self.counter_box * 115
        self.button_pos_list.append([b_x ,b_y - self.remove_counter * 115])
        self.button_list.append(tk.Button(self.frame2,width=7,text='取り消し', command=self.btn_click(self.counter_box)))
        self.button_list[self.counter_box].place(x=b_x, y=b_y - self.remove_counter * 115)
        
        r_x , r_y = 80, 80  + self.counter_box * 115
        self.risk_label_pos_list.append([r_x , r_y - self.remove_counter * 115])
        self.risk_label_list.append(tk.Label(self.frame2, text="リスク項目", width=8))
        self.risk_label_list[self.counter_box].place(x=r_x, y=r_y - self.remove_counter * 115)
        
        stage_label_x , stage_label_y = 80, 30  + self.counter_box * 115
        self.risk_stage_pos_list.append([stage_label_x , stage_label_y - self.remove_counter * 115])
        self.risk_stage_list.append(tk.Label(self.frame2, text="リスク度", width=8))
        self.risk_stage_list[self.counter_box].place(x=stage_label_x, y=stage_label_y - self.remove_counter * 115)
        
        s_x , s_y = 5, 20  + self.counter_box * 115
        self.frame_pos_list.append([s_x , s_y - self.remove_counter * 115])
        self.frame_list.append(self.anno_canvas.create_rectangle(s_x, s_y- self.remove_counter * 115, 
                                                                 s_x + 580, s_y + 110- self.remove_counter * 115, 
                                                                 fill = "pink", tag = "frame_{}".format(self.counter_box)))
        #print("frame_{}".format(self.counter_box))
        #self.frame_list[self.counter_box].place(x=s_x, y=s_y - self.remove_counter * 115)
    
    def btn_click_next(self):
        
        self.image_id = self.files[self.counter_img].split(".")[0]
        self.image_id = int(self.image_id) - 1
        data_exist_list = list(filter(None, self.box_start_list))
        
        l_id = tk.Label(self.frame3,text="image_id:{}".format(self.image_id+1), relief="flat")
        l_id.grid(row=0, column=1, sticky = tk.W)#表示しているimage_idの更新
        
        if data_exist_list != []:
            self.save_json(self.box_start_list, self.box_end_list, self.txt_var_list, self.all_check_var_list, self.all_radio_var_list, self.image_id)#jsonファイルに記録
        
        self.unbind()
        self.data_clear()
        self.counter_img += 1
        self.cap = cv2.VideoCapture('C:/Users/naoki/anaconda_work/Faster-R-CNN-pytorch-main/data/ori_data/drama_clip/{}/clip.mp4'.format(self.counter_img - 1))
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        self.play_video()
        
    def data_clear(self):
        
        
        for i in range(self.counter_box+1):
            
            if self.rect_list[i] == None:
                pass
            else:
                self.canvas.delete(self.rect_list[i])
                self.rect_id_list[i].destroy()
                self.label_list[i].destroy()
                self.risk_label_list[i].destroy()
                self.risk_stage_list[i].destroy()
                self.txt_list[i].destroy()
                self.button_list[i].destroy()
                self.anno_canvas.delete("frame_{}".format(i))
                for item in range(len(self.item)):
                    self.check_list[i][item].destroy()
                for stage in range(len(self.stage)):
                    self.radio_list[i][stage].destroy()
                

        self.remove_counter = 0
        self.counter_box = -1
        
        self.rect_list = []
        self.txt_list = []
        self.button_list = []
        self.txt_var_list = []
        self.check_list = []
        self.radio_list = []
        self.label_list = []
        self.risk_label_list = []
        self.risk_stage_list = []
        
        self.rect_id_list = []
        self.frame_list = []
        self.check_var_list = []
        self.radio_var_list = []
        self.all_check_var_list = []
        self.all_radio_var_list = []
        
        
        self.label_pos_list = []
        self.button_pos_list = []
        self.text_pos_list = []
        self.check_pos_list = []
        self.radio_pos_list = []
        self.risk_label_pos_list = []
        self.risk_stage_pos_list = []
        self.frame_pos_list = []
        
        self.box_start_list = []
        self.box_end_list = []

        self.check_var_dict = {}
        self.txt_var_list = []
       
    def btn_click_re(self):
        self.unbind()
        self.hidden_id()
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        self.play_video()
    
    def hidden_id(self):
        for i in range(len(self.rect_id_list)):
            if self.rect_id_list[i] != None:
                self.rect_id_list[i].place_forget()
    
    def save_json(self, s_rect, e_rect, text, risk, risk_stage, img_id): 
        
        """
        
        btn_click_next()関数から呼び出され、入力されたデータを加工しjsonファイルとして保存する
        
        args
        ---------------
        
        s_rect : list
            各ボックスの左上の座標が格納されたリスト
        e_rect : list
            各ボックスの右下の座標が格納されたリスト
        text : list
            各ボックスのテキストバックスに打ち込まれた文字列に関するオブジェクトが格納されたリスト
        risk_degree : list
            各ボックスに保持されているチェックボックスのチェック可否の情報が含まれるオブジェクトが格納されたリスト
        img_id : int
            画面に表示されている画像の画像ID
        
        """
        
        self.one_region_data = {"regions":[]}
        
        
        
        record_s_rect_data = list(filter(None, s_rect))
        record_e_rect_data = list(filter(None, e_rect))
        record_txt_data = list(filter(None, text))
        record_risk_data = list(filter(None, risk))
        record_stage_data = list(filter(None, risk_stage))
        #print(record_stage_data)
        
        all_box_risk_data = []
        all_box_stage_data = []
        for i in range(len(record_txt_data)):
            record_txt_data[i] = record_txt_data[i].get()
            one_risk_data = []
            one_stage_data = []
            for item in range(len(self.item)):
                one_risk_data.append(1 if record_risk_data[i][item].get()==True else 0)
            
            one_stage_data.append(record_stage_data[i].get())
            all_box_risk_data.append(one_risk_data)
            all_box_stage_data.append(one_stage_data)
        print(all_box_stage_data)        

        for i in range(len(record_s_rect_data)):
            self.region_id += 1
            self.one_region_data["regions"].append({"region_id": self.region_id, 
                                                    "boxes":[record_s_rect_data[i][0], record_s_rect_data[i][1],record_e_rect_data[i][0], record_e_rect_data[i][1]], 
                                                    "image_id": int(img_id), "label": record_txt_data[i], "risk": str(all_box_risk_data[i]), "risk_degree": str(all_box_stage_data[i][0])})
            
            
        self.one_region_data["id"] = int(img_id)
        
        self.region_json_data.append(self.one_region_data)
        
        self.image_json_data.append({"width": self.img_width, "url": "ori_data/{}.jpg".format(img_id), "height": self.img_height, "image_id": int(img_id)})
        
        
        with open(self.ori_data_path + 'region_info.json', 'w') as fp:
            json.dump(self.region_json_data, fp, ensure_ascii=False, indent=2)
            
        with open(self.ori_data_path + 'image_info.json', 'w') as fp:
            json.dump(self.image_json_data, fp, ensure_ascii=False, indent=2)
        
        with open('class_label.json', 'w') as fp:
            json.dump(self.item, fp, ensure_ascii=False)
        
        
def main():
    
    root = tk.Tk()
    app = Application(master=root)
    app.mainloop()

if __name__ == "__main__":
    main()

24.25 41.0
